In [3]:
#imports

import os
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
from numpy import reshape
import re
%matplotlib inline

In [4]:
os.chdir("/Users/andrewcaffey/Documents/Projects/Data/PCPP/parts/UPS/parts/")
part_list = []
comments = []
for i in os.listdir(os.getcwd()):
    a = open(i, 'r')
    #print a.read()
    b = BeautifulSoup(a)

    #basic info
    if b.find('h4', attrs={'class':'kind'}) != None:
        kind = b.find('h4', attrs={'class':'kind'}).text
        part_name = b.find('h1', attrs={'class':'name'}).text
        link = b.find('input', attrs={'name':'url'})['value']
        info_dict = {'Kind':kind, 'Name':part_name, 'Link': link}

        #prices
        if b.find_all('td', attrs={'class':'base'}) != None:
            price_list = b.find_all('td', attrs={'class':'base'})
            price_list = [float(x.text.strip('$')) for x in price_list]
            #average_price = sum(price_list)/len(price_list)
            price_dict = {'Prices':price_list,}

        #specs
        spec_labels = b.find('div', attrs={'class':'specs block'}).find_all('h4')
        spec_labels = [x.contents[0] for x in spec_labels]
        spec_values = str(b.find('div', attrs={'class':'specs block'}))

        vals = [x.strip().split('</h4>')[1].strip('\n').strip() for x in spec_values.split("<h4>")[1:]]
        vals[-1] = vals[-1].split('\n')[0]
        spec_values = vals

        spec_values = spec_values[0:len(spec_labels)+1]
        spec_dict = {spec_label:spec_value for spec_label, spec_value in zip(spec_labels,spec_values)}

        part_dict = dict(spec_dict.items() + info_dict.items() + price_dict.items())
        part_list.append(part_dict)

        #reviews
    reviews = b.find('div', attrs={'class':'part-reviews'})
    if reviews != None:
        reviews = reviews.find_all('div',attrs={'class':'part-review-block'})
        star_list = [len(reviews[x].find('ul',attrs={'class':'stars'}).find_all('li',attrs={'class':'full-star'})) for x in range(len(reviews))]

        comment_text_list = b.find_all('div', attrs={'class':'comment-message markdown'})
        comment_text_list = [comment_text_list[x].find_all('p') for x in range(len(comment_text_list))]

        comment_text_list_clean = []
        for i, x in enumerate(comment_text_list):
            comment = ""
            for y in x:
                try:
                    comment += y.contents[0] + " "
                except:
                    pass
            comment_text_list_clean.append(comment)

        

        review = zip(star_list, comment_text_list_clean)
        
        comments.append(review)

    a.close()
    b.decompose()

In [5]:
df = pd.DataFrame(part_list)

In [6]:
#standard

#average price
df['avg'] = [sum(x)/len(x) if len(x)>0 else 0 for x in df.Prices]

#short_link
df['short_link'] = [x.split('/')[-1] for x in df.Link]

In [7]:
df['load_cap_w'] = df['Load Capacity (W)']
df.load_cap_w = [x if type(x)==str else '0 W' for x in df.load_cap_w]
df.load_cap_w = [float(x.strip('W').strip()) for x in df.load_cap_w]

df['load_cap_va'] = df['Load Capacity (VA)']
df.load_cap_va = [x if type(x)==str else '0 VA' for x in df.load_cap_va]
df.load_cap_va = [float(x.strip('VA').strip()) for x in df.load_cap_va]


df['max_batt_recharge_time'] = df['Maximum Battery Recharge Time']

df.max_batt_recharge_time = [x if type(x)==str else '0 Hour' for x in df.max_batt_recharge_time]
df.max_batt_recharge_time = [float(x.strip('Hour').strip()) for x in df.max_batt_recharge_time]

In [8]:
df['full_load'] = df['Backup/Run Time (Full Load)']

full_load_vals = []
for x in df.full_load:
    if type(x)!=str:
        x = 0
        full_load_vals.append(x)
    elif 'Minutes' in x:
        x = float(x.strip('Minutes').strip())
        full_load_vals.append(x)  
    elif 'Minute' in x:
        x = float(x.strip('Minutes').strip())
        full_load_vals.append(x)
    elif 'Seconds' in x:
        x = float(x.strip('Seconds').strip())/60
        full_load_vals.append(x)
    elif 'Second' in x:
        x = float(x.strip('Second').strip())/60
        full_load_vals.append(x)
    else:
        print x
        
df.full_load = full_load_vals

In [9]:
df['half_load'] = df['Backup/Run Time (Half Load)']

half_load_vals = []
for x in df.half_load:
    if type(x)!=str:
        x = 0
        half_load_vals.append(x)
    elif 'Minutes' in x:
        x = float(x.strip('Minutes').strip())
        half_load_vals.append(x)
        
    elif 'Minute' in x:
        x = float(x.strip('Minutes').strip())
        half_load_vals.append(x)
    elif 'Seconds' in x:
        x = float(x.strip('Seconds').strip())/60
        half_load_vals.append(x)
    elif 'Second' in x:
        x = float(x.strip('Second').strip())/60
        half_load_vals.append(x)
    else:
        print x
        
df.half_load = half_load_vals

In [13]:
df.to_csv('ups_csv_last_final.csv', encoding='utf-8', index=False)

In [11]:
stars = [x[0][0] for x in comments[:]]
text = [x[0][1] for x in comments[:]]
ups_part_reviews = pd.DataFrame({'Stars':stars, 'Comments':text, 'Type':'UPS'})

In [12]:
ups_part_reviews.to_csv('ups_part_reviews.csv', encoding='utf-8', index=False)